In [1]:
from complementary_models import *
import numpy as np 
from complementary_models import infer_nonsta_dir
import torch

Global seed set to 0


In [2]:
A = np.random.rand(5,5)
B = np.random.rand(5,3)
C = np.random.rand(5,1)


In [3]:
AT = torch.tensor(A)
BT = torch.tensor(B)
CT = torch.tensor(C)


In [60]:
X = A
Y = B
c_indx = C


width = 0.1
T,d = X.shape
## normalization
X = X - np.matlib.repmat( np.mean(X,0), X.shape[0], 1)
X = np.matmul(X, np.diag(1./np.std(X, axis=0, ddof=1)))
## normalization
Y = Y - np.matlib.repmat( np.mean(Y,0), Y.shape[0], 1)
Y = np.matmul(Y, np.diag(1/np.std(Y, axis=0,  ddof=1)))
theta = 1/width**2
Lambda = 1
Wt = 1
theta = 1/width**2
Kxx, _ = kernel(X,X, [theta,1])
Kxx , _ = kernel(X, X, [theta,1])
Kyy, _ = kernel(Y, Y, [theta,1])
Ktt, _ = kernel(c_indx, c_indx, [1/Wt**2,1])


invK = pdinv( np.multiply(Kxx,Ktt) +  Lambda * np.eye(T))

Kxx3 = np.linalg.matrix_power(Kxx,3)
prod_invK =  np.matmul(np.matmul(invK, Kyy), invK)
###now finding Ml
Ml = np.matmul(np.matmul(1/T**2*Ktt, np.multiply(Kxx3, prod_invK)), Ktt)
# print(Ml)
###the square distance
D = np.matmul(np.diag(np.diagonal(Ml)),np.ones(Ml.shape)) + np.matmul(np.ones(Ml.shape), np.diag(np.diagonal(Ml))) - 2*Ml
# print(D)
DD = D[np.where(np.tril(np.ones(D.shape),-1) != 0)]
sigma2_square = np.median(np.array(DD))
Mg = np.exp(-D/sigma2_square/2)
Mg = np.exp(-D/sigma2_square/2)

print("sorted DD:{}".format(sorted(DD)))
print("sigma2_square {}".format(sigma2_square))
print("D {}".format(D))
print("-D/sigma2_square/2 {}".format(-D/sigma2_square/2))
print(Mg)

sorted DD:[matrix([[7.18173560e-05, 2.23331022e-03, 1.75216607e-03, 9.86906937e-04,
         5.92728282e-04, 3.86995593e-04, 2.34054014e-04, 5.48077455e-04,
         3.02627781e-03, 1.84070990e-03]])]
sigma2_square 0.000789817609334692
D [[0.00000000e+00 7.18173560e-05 2.23331022e-03 9.86906937e-04
  2.34054014e-04]
 [7.18173560e-05 0.00000000e+00 1.75216607e-03 5.92728282e-04
  5.48077455e-04]
 [2.23331022e-03 1.75216607e-03 0.00000000e+00 3.86995593e-04
  3.02627781e-03]
 [9.86906937e-04 5.92728282e-04 3.86995593e-04 0.00000000e+00
  1.84070990e-03]
 [2.34054014e-04 5.48077455e-04 3.02627781e-03 1.84070990e-03
  0.00000000e+00]]
-D/sigma2_square/2 [[-0.         -0.04546452 -1.4138139  -0.62476889 -0.14816966]
 [-0.04546452 -0.         -1.10922196 -0.37523111 -0.34696457]
 [-1.4138139  -1.10922196 -0.         -0.24499048 -1.91580802]
 [-0.62476889 -0.37523111 -0.24499048 -0.         -1.1652753 ]
 [-0.14816966 -0.34696457 -1.91580802 -1.1652753  -0.        ]]
[[1.         0.95555351 0.

In [61]:
invK2 = pdinv(Ktt +  Lambda * np.eye(T))
# print(invK2)
Ml2 = np.matmul(Ktt, invK2)
Ml2 = np.matmul(Ml2, Kxx)
Ml2 = np.matmul(Ml2, invK2)
Ml2 = np.matmul(Ml2, Ktt)
D2 =np.matmul(np.diag(np.diagonal(Ml2)),np.ones(Ml2.shape)) + np.matmul(np.ones(Ml2.shape), np.diag(np.diagonal(Ml2))) - 2*Ml2
# print(D2)
# % Gaussian kernel
DD2 = D2[np.where(np.tril(np.ones(D2.shape),-1) != 0)]
sigma2_square2 = np.median(np.array(DD2))
print(sorted(DD2))
# sigma2_square2 = median( D2(find(tril(ones(size(D2)),-1))) );
Mg2 = np.exp(-D2/sigma2_square2/2)
print(D2)
print()
print(-D/sigma2_square/2)
print(Mg2)
# %% 
H = np.eye(T)-1/T*np.ones([T,T])
Mg = np.matmul(np.matmul(H, Mg), H)
Mg2 = np.matmul(np.matmul(H, Mg2), H)
testStat = 1/T**2*np.sum(np.sum(np.multiply(Mg, Mg2)))


[matrix([[0.00287762, 0.09240317, 0.06364913, 0.04657748, 0.026573  ,
         0.00833232, 0.00596218, 0.01702842, 0.14164172, 0.08432525]])]
[[0.         0.00287762 0.09240317 0.04657748 0.00596218]
 [0.00287762 0.         0.06364913 0.026573   0.01702842]
 [0.09240317 0.06364913 0.         0.00833232 0.14164172]
 [0.04657748 0.026573   0.00833232 0.         0.08432525]
 [0.00596218 0.01702842 0.14164172 0.08432525 0.        ]]

[[-0.         -0.04546452 -1.4138139  -0.62476889 -0.14816966]
 [-0.04546452 -0.         -1.10922196 -0.37523111 -0.34696457]
 [-1.4138139  -1.10922196 -0.         -0.24499048 -1.91580802]
 [-0.62476889 -0.37523111 -0.24499048 -0.         -1.1652753 ]
 [-0.14816966 -0.34696457 -1.91580802 -1.1652753  -0.        ]]
[[1.         0.96142528 0.28274978 0.52901676 0.92172751]
 [0.96142528 1.         0.41890452 0.69540225 0.79232292]
 [0.28274978 0.41890452 1.         0.89234126 0.14423574]
 [0.52901676 0.69540225 0.89234126 1.         0.31576272]
 [0.92172751 0.792

In [63]:
X = AT
Y = BT
c_indx = CT

width = 0.1
T,d = X.shape
## normalization
X = X - torch.mean(X, 0)
X = X / torch.std(X, axis=0, unbiased=True)

# X = torch.matmul(X, np.diag(1./np.std(X, axis=0, ddof=1)))
## normalization
Y = Y - torch.mean(Y, 0)
Y = Y / torch.std(Y, axis=0, unbiased=True)

theta = 1/width**2
Lambda = 1
Wt = 1
theta = 1/width**2
Kxx, _ = torch_kernel(X,X, [theta,1])
Kxx , _ = torch_kernel(X, X, [theta,1])
Kyy, _ = torch_kernel(Y, Y, [theta,1])
Ktt, _ = torch_kernel(c_indx, c_indx, [1/Wt**2,1])


invK = torch.inverse(torch.mul(Kxx,Ktt) +  Lambda * torch.eye(T))

Kxx3 = torch.matmul(torch.matmul(Kxx, Kxx),Kxx)
prod_invK =  torch.matmul(torch.matmul(invK, Kyy), invK)
###now finding Ml
Ml = torch.matmul(torch.matmul(1/T**2*Ktt, torch.mul(Kxx3, prod_invK)), Ktt)
# print(Ml)
###the square distance
D = torch.matmul(torch.diag(torch.diag(Ml)),torch.ones(Ml.shape, dtype=float)) + torch.matmul(torch.ones(Ml.shape, dtype=float), torch.diag(torch.diag(Ml))) - 2*Ml
# print(D)
DD = D[torch.where(torch.tril(torch.ones(D.shape, dtype=float),-1) != 0)]
sigma2_square = torch.quantile(DD, q = 0.5)
Mg = torch.exp(-D/sigma2_square/2.0)
# print(D.numpy().astype(np.float64))
print("sorted DD:{}".format(sorted(DD)))
print("sigma2_square {}".format(sigma2_square))
print("D {}".format(D))
print("-D/sigma2_square/2 {}".format(-D/sigma2_square/2))
print(Mg)

sorted DD:[tensor(7.1817e-05, dtype=torch.float64), tensor(0.0002, dtype=torch.float64), tensor(0.0004, dtype=torch.float64), tensor(0.0005, dtype=torch.float64), tensor(0.0006, dtype=torch.float64), tensor(0.0010, dtype=torch.float64), tensor(0.0018, dtype=torch.float64), tensor(0.0018, dtype=torch.float64), tensor(0.0022, dtype=torch.float64), tensor(0.0030, dtype=torch.float64)]
sigma2_square 0.0007898176093347475
D tensor([[0.0000e+00, 7.1817e-05, 2.2333e-03, 9.8691e-04, 2.3405e-04],
        [7.1817e-05, 0.0000e+00, 1.7522e-03, 5.9273e-04, 5.4808e-04],
        [2.2333e-03, 1.7522e-03, 0.0000e+00, 3.8700e-04, 3.0263e-03],
        [9.8691e-04, 5.9273e-04, 3.8700e-04, 0.0000e+00, 1.8407e-03],
        [2.3405e-04, 5.4808e-04, 3.0263e-03, 1.8407e-03, 0.0000e+00]],
       dtype=torch.float64)
-D/sigma2_square/2 tensor([[-0.0000, -0.0455, -1.4138, -0.6248, -0.1482],
        [-0.0455, -0.0000, -1.1092, -0.3752, -0.3470],
        [-1.4138, -1.1092, -0.0000, -0.2450, -1.9158],
        [-0.62

In [29]:
invK2 = torch.inverse(Ktt +  Lambda * torch.eye(T))
# print(invK2)
Ml2 = torch.matmul(Ktt, invK2)
Ml2 = torch.matmul(Ml2, Kxx)
Ml2 = torch.matmul(Ml2, invK2)
Ml2 = torch.matmul(Ml2, Ktt)
D2 =torch.matmul(torch.diag(torch.diag(Ml2)),torch.ones(Ml2.shape, dtype=float)) + torch.matmul(torch.ones(Ml2.shape, dtype=float), torch.diag(torch.diag(Ml2))) - 2*Ml2
# print(D2)

# % Gaussian kernel
DD2 = D2[torch.where(torch.tril(torch.ones(D2.shape),-1) != 0)]
sigma2_square2 = torch.quantile(DD2, q=0.5)

# sigma2_square2 = median( D2(find(tril(ones(size(D2)),-1))) );
print(-D2/sigma2_square2/2)
Mg2 = torch.exp(-D2/sigma2_square2/2)
print(Mg2)

# %% 
H = torch.eye(T)-1/T*torch.ones([T,T],dtype=float)
Mg = torch.matmul(torch.matmul(H, Mg), H)
Mg2 = torch.matmul(torch.matmul(H, Mg2), H)
testStat = 1/T**2*torch.sum(torch.sum(torch.mul(Mg, Mg2)))

tensor([[-0.0000, -0.2978, -0.1100, -0.4335, -1.8217],
        [-0.2978, -0.0000, -0.7618, -0.0129, -0.6706],
        [-0.1100, -0.7618, -0.0000, -0.9682, -2.7590],
        [-0.4335, -0.0129, -0.9682, -0.0000, -0.5000],
        [-1.8217, -0.6706, -2.7590, -0.5000, -0.0000]], dtype=torch.float64)
tensor([[1.0000, 0.7424, 0.8959, 0.6483, 0.1617],
        [0.7424, 1.0000, 0.4668, 0.9872, 0.5114],
        [0.8959, 0.4668, 1.0000, 0.3798, 0.0634],
        [0.6483, 0.9872, 0.3798, 1.0000, 0.6065],
        [0.1617, 0.5114, 0.0634, 0.6065, 1.0000]], dtype=torch.float64)


In [4]:
torch_infer_nonsta_dir(AT, BT, CT)

(tensor(0.0638, dtype=torch.float64),
 tensor([[ 0.2010,  0.1465, -0.1765, -0.0295, -0.1413],
         [ 0.1465,  0.1497, -0.0823, -0.1684, -0.0454],
         [-0.1765, -0.0823,  0.2958, -0.2958,  0.2588],
         [-0.0295, -0.1684, -0.2958,  0.8028, -0.3091],
         [-0.1413, -0.0454,  0.2588, -0.3091,  0.2370]], dtype=torch.float64),
 tensor([[ 0.1842,  0.1182, -0.2056,  0.0787, -0.1755],
         [ 0.1182,  0.1209, -0.0794, -0.1087, -0.0510],
         [-0.2056, -0.0794,  0.3335, -0.3497,  0.3012],
         [ 0.0787, -0.1087, -0.3497,  0.7329, -0.3531],
         [-0.1755, -0.0510,  0.3012, -0.3531,  0.2784]], dtype=torch.float64))

In [5]:
infer_nonsta_dir(A, B, C)

(0.063826910390044,
 matrix([[ 0.20095125,  0.14647265, -0.1765497 , -0.02954141, -0.14133279],
         [ 0.14647265,  0.14971551, -0.08234002, -0.16844106, -0.04540708],
         [-0.1765497 , -0.08234002,  0.29580064, -0.2957532 ,  0.25884227],
         [-0.02954141, -0.16844106, -0.2957532 ,  0.80280603, -0.30907036],
         [-0.14133279, -0.04540708,  0.25884227, -0.30907036,  0.23696795]]),
 matrix([[ 0.18419595,  0.11820023, -0.20558437,  0.0786597 , -0.17547151],
         [ 0.11820023,  0.12091152, -0.07940384, -0.10873531, -0.0509726 ],
         [-0.20558437, -0.07940384,  0.33345128, -0.34968609,  0.30122302],
         [ 0.0786597 , -0.10873531, -0.34968609,  0.73290387, -0.35314217],
         [-0.17547151, -0.0509726 ,  0.30122302, -0.35314217,  0.27836327]]))